In [ ]:

import os
import datetime
import numpy as np
from scipy.io.wavfile import write
import sounddevice as sd
import ipywidgets as widgets
from IPython.display import display
import threading
import librosa
import matplotlib.pyplot as plt
import librosa.display


fs = 16000
recording = []
is_recording = False

save_folder = "recordings"

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

spect_folder = "spectrograms"
if not os.path.exists(spect_folder):
    os.makedirs(spect_folder)
output = widgets.Output()


def record_thread():
    global recording, is_recording
    recording = []
    def callback(indata, frames, time, status):
        if is_recording:
            recording.append(indata.copy())
    with sd.InputStream(samplerate=fs, channels=1, callback=callback):
        while is_recording:
            sd.sleep(100)

def save_spectrogram(audio_file, spect_folder):
    y, sr = librosa.load(audio_file, sr=fs)
    
    S = librosa.feature.melspectrogram(
        y=y, 
        sr=sr, 
        n_fft=1024,
        hop_length=256,
        n_mels=128,
        power=2.0
    )
    # Use top_db to clip the dynamic range
    S_dB = librosa.power_to_db(S, ref=np.max, top_db=80)
    
    # Normalize to use full color range
    S_dB = (S_dB - S_dB.min()) / (S_dB.max() - S_dB.min()) * 255

    h, w = S_dB.shape
    dpi = 100
    fig_w = w / dpi
    fig_h = h / dpi
    
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    ax.imshow(S_dB, origin='lower', aspect='auto', cmap='magma', vmin=0, vmax=255)
    
    base_filename = os.path.splitext(os.path.basename(audio_file))[0]
    spect_filename = os.path.join(spect_folder, f"{base_filename}.png")
    plt.savefig(spect_filename, dpi=dpi)
    plt.close()
    
    return spect_filename

def toggle_recording(b):
    global is_recording, recording
    with output:
        output.clear_output()
        if not is_recording:
            is_recording = True
            button.description = "Stop Recording"
            print("Recording...")
            threading.Thread(target=record_thread, daemon=True).start()
        else:
            is_recording = False
            button.description = "Start Recording"
            audio_data = np.concatenate(recording, axis=0)
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = os.path.join(save_folder, f"recording_{timestamp}.wav")
            write(filename, fs, np.int16(audio_data * 32767))
            print(f"Recording is finished! Audio is saved as: {filename}")
            spect_filename = save_spectrogram(filename, spect_folder)
            print(f"Spectrogram saved as: {spect_filename}")  

button = widgets.Button(description="Start Recording", button_style='success')
button.on_click(toggle_recording)

display(button, output)

Button(button_style='success', description='Start Recording', style=ButtonStyle())

Output()